### MNIST Handwritten Digit Recognizer With Live Video Feed
<pre>
Authors            : Rajendra Bhagroo  | Christopher Guevarra
Emails             : Rbhagroo@Nyit.edu | Cguevarr@nyit.edu

Course Name        : Artificial Intelligence 1
Course Number      : CSCI 355 - W01D
Course Instructor  : Maherukh Akhtar

Date Due           : 12/10/18
Date Last Modified : 11/18/18
Kernel             : Python 3.6.3
</pre>

In [ ]:
# Install pip packages for the current Jupyter kernel
import sys

!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install keras

# Python 3.6 Compatable: https://www.tensorflow.org/install/pip
!{sys.executable} -m pip install --upgrade tensorflow

In [ ]:
# Sets matplotlib's backend engine to display plots within current Jupyter kernel
%matplotlib inline

# Sets Tensorflow as backend engine
import tensorflow as tf

import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# To implement different dataset see: http://yann.lecun.com/exdb/mnist/
from keras.datasets import mnist

In [ ]:
# Load Data

""""
Dataset Info:
    ...
""""
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess Data
# Normalize Data 
# Reshape Data
# Plot Data
# Build Model
# Evaluate Model
# Save Model

# OpenCV